In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init


import torchvision.datasets as datasets
import torchvision.transforms as transforms


from torch.utils.data import DataLoader

import numpy as np 
import matplotlib.pyplot as plt

In [5]:
batch_size = 1000
learning_rate = 0.0002
num_epochs = 10

In [6]:
mnist_train = datasets.MNIST(root="../Data/", train=True, transform=transforms.ToTensor(), target_transform=None, download=True)
mnist_test = datasets.MNIST(root="../Data/", train=False, transform=transforms.ToTensor(), target_transform=None, download=True)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:22<00:00, 441327.18it/s] 


Extracting ../Data/MNIST/raw/train-images-idx3-ubyte.gz to ../Data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 143640.92it/s]


Extracting ../Data/MNIST/raw/train-labels-idx1-ubyte.gz to ../Data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:06<00:00, 265173.48it/s]


Extracting ../Data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../Data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 4092487.38it/s]

Extracting ../Data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../Data/MNIST/raw



In [9]:
train_loader = DataLoader(mnist_train, batch_size = batch_size, shuffle = True, num_workers = 2, drop_last = True)
test_loader = DataLoader(mnist_test, batch_size = batch_size, shuffle = False, num_workers = 2, drop_last = True)

In [29]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
    
        self.layer = nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5),
            nn.ReLU(),
            nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5),
            nn.ReLU(),
    
            nn.MaxPool2d(kernel_size=2,stride=2),
    
            nn.Conv2d(in_channels=32,out_channels=64,kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(64*3*3,100),
            nn.ReLU(),
            nn.Linear(100,10)
        )
    def forward(self,x):
        out = self.layer(x)
        out = out.view(batch_size,-1)
        out = self.fc_layer(out)
        return out 

In [30]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [31]:
model = CNN().to(device)

In [32]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

In [33]:
loss_arr = []
for i in range(num_epochs):
    for j,[image,label] in enumerate(train_loader):
        x = image.to(device)
        y = label.to(device)

        optimizer.zero_grad()

        output = model.forward(x)

        loss = loss_func(output,y)
        loss.backward()
        optimizer.step()

        if j % 1000 == 0:
            print(loss)
            loss_arr.append(loss.cpu().detach().numpy())

tensor(2.3054, grad_fn=<NllLossBackward0>)
tensor(0.7188, grad_fn=<NllLossBackward0>)
tensor(0.3566, grad_fn=<NllLossBackward0>)
tensor(0.2420, grad_fn=<NllLossBackward0>)
tensor(0.2354, grad_fn=<NllLossBackward0>)
tensor(0.1502, grad_fn=<NllLossBackward0>)
tensor(0.1496, grad_fn=<NllLossBackward0>)
tensor(0.1163, grad_fn=<NllLossBackward0>)
tensor(0.0821, grad_fn=<NllLossBackward0>)
tensor(0.0916, grad_fn=<NllLossBackward0>)


In [34]:
correct = 0 
total = 0

model.eval()

with torch.no_grad():
    for image,label in test_loader:
        x = image.to(device)
        y = label.to(device)

        output = model.forward(x)

        _,output_index = torch.max(output,1)

        total += label.size(0)

        correct += (output_index == y).sum().float()

    print("Accuracy of Test Data {}%".format(100*correct/total))

Accuracy of Test Data 97.79000091552734%
